In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import SkyScan
import sky as sk
import pandas as pd
import requests
import json
from pandas.io.json import json_normalize #package for flattening json in pandas df

In [ ]:
# Use the endpoint POST Create session to create a query session for LIVE FLIGHT SEARCH
# Required: originPlace: 
#           destinationPlace: 
#           outboundDate:
    
#     Get back the session ID in the Location header from POST Create session.
#     Poll the results using the endpoint GET Poll session results.

In [ ]:
# POST to initiate LIVE FLIGHT SEARCH

post_url = sky_api_host + "apiservices/pricing/v1.0"

payload = "inboundDate=&cabinClass=&children=&infants=&country=&currency=&locale=&originPlace=&destinationPlace=&outboundDate=&adults="
headers = {
    'x-rapidapi-host': sky_api_host,
    'x-rapidapi-key': sk.sky_key,
    'content-type': "application/x-www-form-urlencoded"
    }

response = requests.request("POST", post_url, data=payload, headers=headers)

In [3]:
# Find Location IDs based on string searches
test_loc = SkyScan.get_locations('New',sk.key)
test_loc['PlaceId'] # use these as inputs

0    NYCA-sky
1     JFK-sky
2     EWR-sky
3     LGA-sky
4     SWF-sky
5     DEL-sky
6    MSYA-sky
7     MSY-sky
8     NEW-sky
9     HVN-sky
Name: PlaceId, dtype: object

In [149]:
# BROWSE QUOTES
test_quote = SkyScan.browse_quotes('SFO-sky','JFK-sky', sk.key, '2019-09-01')
t_quote = json.loads(test_quote.text)
quotes = SkyScan.parse_quotes(test_quote)  
quotes

,Direct,MinPrice,OutboundLeg.CarrierIds,OutboundLeg.DepartureDate,OutboundLeg.DestinationId,OutboundLeg.OriginId,QuoteDateTime,QuoteId,CarrierNames
0,False,261.0,[833],2019-09-01T00:00:00,60987,81727,2019-08-21T23:43:00,1,American Airlines
1,True,305.0,[870],2019-09-01T00:00:00,60987,81727,2019-08-14T02:55:00,2,jetBlue


In [154]:
# BROWSE ROUTES
test_route = SkyScan.browse_routes('SFO-sky','JFK-sky', sk.key, '2019-09-01')
t_route = json.loads(test_route.text)
print(t_route.keys())
[type(t_route[key]) for key in t_route.keys()]

ConnectionError: HTTPSConnectionPool(host='skyscanner-skyscanner-flight-search-v1.p.rapidapi.com', port=443): Max retries exceeded with url: /apiservices/browseroutes/v1.0/US/USD/en-US/SFO-sky/JFK-sky/2019-09-01 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000012EDA4B6E10>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [153]:
pd.DataFrame(t_route['Routes'])

""


In [8]:
# BROWSE DATES
test_dates = SkyScan.browse_dates('EWR-sky','LAX-sky', sk.key, 'anytime')
t_dates = json.loads(test_dates.text)
print(t_dates.keys())
[type(t_dates[key]) for key in t_dates.keys()]

dict_keys(['Dates', 'Quotes', 'Places', 'Carriers', 'Currencies'])


[dict, list, list, list, list]

In [14]:
[pd.DataFrame(t_dates[key]) for key in t_dates.keys()]

[                                        OutboundDates
 0   {'PartialDate': '2019-08', 'QuoteIds': [1, 2],...
 1   {'PartialDate': '2019-09', 'QuoteIds': [3, 4],...
 2   {'PartialDate': '2019-10', 'QuoteIds': [5, 6],...
 3   {'PartialDate': '2019-11', 'QuoteIds': [7, 8],...
 4   {'PartialDate': '2019-12', 'QuoteIds': [9, 10]...
 5   {'PartialDate': '2020-01', 'QuoteIds': [11], '...
 6   {'PartialDate': '2020-02', 'QuoteIds': [12], '...
 7   {'PartialDate': '2020-03', 'QuoteIds': [13], '...
 8   {'PartialDate': '2020-04', 'QuoteIds': [14], '...
 9   {'PartialDate': '2020-05', 'QuoteIds': [15], '...
 10  {'PartialDate': '2020-06', 'QuoteIds': [16], '...
 11  {'PartialDate': '2020-07', 'QuoteIds': [17], '...,
     Direct  MinPrice                                        OutboundLeg  \
 0    False      69.0  {'CarrierIds': [1467], 'OriginId': 50290, 'Des...   
 1     True     236.0  {'CarrierIds': [851], 'OriginId': 50290, 'Dest...   
 2    False      69.0  {'CarrierIds': [1467], 'OriginId'